<a href="https://www.kaggle.com/code/abd0kamel/2023-10-sprint2-kaggle?scriptVersionId=147680985" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# WLASL Dataset Sign Language Model

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.display import clear_output

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        clear_output(True)

print('Completed')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Completed


In [2]:
import json

# Load the JSON file
with open('/kaggle/input/wlasl-processed/WLASL_v0.3.json', 'r') as json_file:
    data = json.load(json_file)

# Words to filter by
words_to_filter = ["where","hello","thank you", "go","stop","here","traffic","good","bad","today"]

# Filter the data
filtered_data = [item for item in data if item.get("gloss") in words_to_filter]

# Extract the first 5 "bbox" elements from each item
for item in filtered_data:
    item["instances"] = item["instances"][:5]

# Save the filtered data with the first 5 "bbox" elements to a JSON file
with open('filtered_data_with_5_bbox.json', 'w') as output_file:
    json.dump(filtered_data, output_file, indent=4)



In [3]:
import os

input_folder = '/kaggle/input/wlasl-processed/videos'

In [4]:
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
print(len(video_files)) 

11980


In [5]:
import json

In [6]:
with open('/kaggle/working/filtered_data_with_5_bbox.json', 'r') as f:
    data = json.load(f)

In [7]:
gloss_list = [] # done
video_name_list = [] # done
start_frame = [] # done
end_frame = [] # done
vid_dir = '/kaggle/input/wlasl-processed/videos'
progress = 0

for word in data:
    for i in range(5):
        vid_name = os.path.join(vid_dir, word['instances'][i]['video_id'])
        start = word['instances'][0]['frame_start']
        end = word['instances'][0]['frame_end']
        label = word['gloss']

        video_name_list.append(vid_name)
        gloss_list.append(label)
        start_frame.append(start)
        end_frame.append(end)

        progress += 1
        print(f'Progress: {progress} / {len(data)*5}')

Progress: 1 / 50
Progress: 2 / 50
Progress: 3 / 50
Progress: 4 / 50
Progress: 5 / 50
Progress: 6 / 50
Progress: 7 / 50
Progress: 8 / 50
Progress: 9 / 50
Progress: 10 / 50
Progress: 11 / 50
Progress: 12 / 50
Progress: 13 / 50
Progress: 14 / 50
Progress: 15 / 50
Progress: 16 / 50
Progress: 17 / 50
Progress: 18 / 50
Progress: 19 / 50
Progress: 20 / 50
Progress: 21 / 50
Progress: 22 / 50
Progress: 23 / 50
Progress: 24 / 50
Progress: 25 / 50
Progress: 26 / 50
Progress: 27 / 50
Progress: 28 / 50
Progress: 29 / 50
Progress: 30 / 50
Progress: 31 / 50
Progress: 32 / 50
Progress: 33 / 50
Progress: 34 / 50
Progress: 35 / 50
Progress: 36 / 50
Progress: 37 / 50
Progress: 38 / 50
Progress: 39 / 50
Progress: 40 / 50
Progress: 41 / 50
Progress: 42 / 50
Progress: 43 / 50
Progress: 44 / 50
Progress: 45 / 50
Progress: 46 / 50
Progress: 47 / 50
Progress: 48 / 50
Progress: 49 / 50
Progress: 50 / 50


gloss_list = [] # done
video_name_list = [] # done
start_frame = [] # done
end_frame = [] # done

In [8]:
print(f'number of words = {len(gloss_list)}\nnumber of videos = {len(video_name_list)}')

number of words = 50
number of videos = 50


In [9]:
import numpy as np

# Specify the file path to your .npy file
file_path = "/kaggle/input/opencvtest/thanks.npy"

# Load the .npy file
loaded_data = np.load(file_path)


In [10]:
loaded_data[0].shape

(543, 3)

In [11]:
import cv2
import numpy as np

def draw(input_path, output_path, npy_file, start_frame=0, end_frame=-1):

    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    landmarks_data = np.load(npy_file)
    frame_index = 0
    
    while cap.isOpened() and frame_index != end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_index >= start_frame:
            landmarks_frame = landmarks_data[frame_index]
            landmarks = [(int(x * width), int(y * height)) for x, y, z in landmarks_frame]

            for x, y in landmarks:
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

            out.write(frame)
        
        frame_index += 1

    cap.release()
    out.release()

In [12]:
import os

input_video = '/kaggle/input/opencvtest/mCjHYreiZ24.mp4'
output_video = '/kaggle/working/output_2.mp4'
npy_path = "/kaggle/input/opencvtest/thanks.npy"

draw(input_video, output_video, npy_path, 0, 89)